## Regression on NFL Game Data

The goal of this project is to use NFL game data in order to predict the scores of both teams in a given matchup. The dataset used comes from [FiveThirtyEight](https://github.com/fivethirtyeight/nfl-elo-game). I took two approaches towards this end. The first was to just use the team information and the provided elo and the second was to use this plus adding in information about the year and location. I considered calculating win loss percentage for a given season up to that point, but the provided elo accounts for that and much more assuming that the elo here is similar to that of elo in chess. 

In [1]:
%matplotlib inline

import pandas
import numpy as np
from tqdm.notebook import tqdm
import pickle

def sklearn_save_model(model, name):
    with open(f"../models/{name}.pkl", 'wb') as f:
        pickle.dump(model, f)

tqdm.pandas()

## Summary of dataset

As you can see below, the dataset has a row for each matchup with information about which teams were playing, when the game took place, the elo of the teams, and wheter the game was a playoff or at a neutral location as well as the final scores of the game.

In [2]:
raw_games = pandas.read_csv('../data/nfl_games.csv')

raw_games.head()

,date,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1
0,1920-09-26,1920,0,0,RII,STP,1503.947,1300.000,0.824651,48,0,1.0
1,1920-10-03,1920,0,0,AKR,WHE,1503.420,1300.000,0.824212,43,0,1.0
2,1920-10-03,1920,0,0,RCH,ABU,1503.420,1300.000,0.824212,10,0,1.0
3,1920-10-03,1920,0,0,DAY,COL,1493.002,1504.908,0.575819,14,0,1.0
4,1920-10-03,1920,0,0,RII,MUN,1516.108,1478.004,0.644171,45,0,1.0


In [3]:
raw_games.describe()

,season,neutral,playoff,elo1,elo2,elo_prob1,score1,score2,result1
count,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000
mean,1982.437569,0.005223,0.034779,1502.458394,1498.918375,0.584829,21.544058,18.578161,0.580681
std,25.448049,0.072084,0.183226,105.015371,104.541271,0.175302,11.289422,10.794566,0.488551
min,1920.000000,0.000000,0.000000,1119.595000,1156.551000,0.070953,0.000000,0.000000,0.000000
25%,1967.000000,0.000000,0.000000,1429.242750,1425.864750,0.461231,14.000000,10.000000,0.000000
50%,1987.000000,0.000000,0.000000,1504.015000,1500.185000,0.596354,21.000000,17.000000,1.000000
75%,2003.000000,0.000000,0.000000,1578.071500,1575.753000,0.719930,28.000000,26.000000,1.000000
max,2018.000000,1.000000,1.000000,1839.663000,1849.484000,0.970516,72.000000,73.000000,1.000000


## Teams and Elo

The first combination of feautres I tried was using the teams and the elo of the teams at the time of that matchup. I one hot encoded the teams as well as scaled the elos and scores.

In [4]:
from sklearn.preprocessing import OneHotEncoder

games = raw_games.copy()

enc = OneHotEncoder(sparse=False)

games['team1'] = enc.fit_transform(np.array(raw_games.team1).reshape(-1, 1)).tolist()
games['team2'] = enc.fit_transform(np.array(raw_games.team2).reshape(-1, 1)).tolist()

games.head()

,date,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1
0,1920-09-26,1920,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1503.947,1300.000,0.824651,48,0,1.0
1,1920-10-03,1920,0,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1503.420,1300.000,0.824212,43,0,1.0
2,1920-10-03,1920,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1503.420,1300.000,0.824212,10,0,1.0
3,1920-10-03,1920,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1493.002,1504.908,0.575819,14,0,1.0
4,1920-10-03,1920,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1516.108,1478.004,0.644171,45,0,1.0


In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

games['elo1'] = scaler.fit_transform(np.array(games.elo1).reshape(-1, 1))
games['elo2'] = scaler.fit_transform(np.array(games.elo2).reshape(-1, 1))

games.head()

,date,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1
0,1920-09-26,1920,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.533772,0.207017,0.824651,48,0,1.0
1,1920-10-03,1920,0,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.533040,0.207017,0.824212,43,0,1.0
2,1920-10-03,1920,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.533040,0.207017,0.824212,10,0,1.0
3,1920-10-03,1920,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.518572,0.502728,0.575819,14,0,1.0
4,1920-10-03,1920,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.550660,0.463902,0.644171,45,0,1.0


In [6]:
from sklearn.preprocessing import MinMaxScaler

score_scaler = MinMaxScaler()

games['score1'] = score_scaler.fit_transform(np.array(games.score1).reshape(-1, 1))
games['score2'] = score_scaler.fit_transform(np.array(games.score2).reshape(-1, 1))

sklearn_save_model(score_scaler, "score_scaler")

games.head()

/home/grant/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/grant/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


,date,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1
0,1920-09-26,1920,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.533772,0.207017,0.824651,0.666667,0.0,1.0
1,1920-10-03,1920,0,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.533040,0.207017,0.824212,0.597222,0.0,1.0
2,1920-10-03,1920,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.533040,0.207017,0.824212,0.138889,0.0,1.0
3,1920-10-03,1920,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.518572,0.502728,0.575819,0.194444,0.0,1.0
4,1920-10-03,1920,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.550660,0.463902,0.644171,0.625000,0.0,1.0


In [7]:
games.score2.describe()

count    16274.000000
mean         0.254495
std          0.147871
min          0.000000
25%          0.136986
50%          0.232877
75%          0.356164
max          1.000000
Name: score2, dtype: float64

### Preparing data for input to models

To prepared the data for the models I combined all the input features into a numpy array and both the about scores into another numpy array.

In [8]:
from sklearn.utils import shuffle

X = np.array(games.team1.tolist())
Z = np.array(games.team2.tolist())

X = np.append(X, Z, 1)

X = np.append(X, np.array(games.elo1).reshape(-1, 1), 1)
X = np.append(X, np.array(games.elo2).reshape(-1, 1), 1)

Y = np.array(games.score1).reshape(-1, 1)
Y = np.append(Y, np.array(games.score2).reshape(-1, 1), 1)

X, Y = shuffle(X, Y)

### Linear Regression

As a good baseline I first used Linear Regression. As you can see from the R^2 scores from the 10-folds, this model seems so bad that I think something is going very wrong during the fitting process or with how the score is calculated. 

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)

fold = 1
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    
    reg = LinearRegression().fit(X_train, Y_train)
    
    sklearn_save_model(reg, f"one_hot_normalized_linear_{fold}")
    
    print(fold, reg.score(X_test, Y_test))
    fold += 1

1 -2.3556757794891936e+22
2 -1.2813286760886175e+24
3 -8.623826997932493e+21
4 -3.781554865051475e+21
5 -3.8476829666876536e+21
6 -2.7702455944179493e+21
7 -3.382216177749537e+21
8 -1.8135746704206677e+22
9 -4.912470766243301e+21
10 -8.382918498596444e+21


### Ridge Regression

Next, I tried using Ridge regression since it's another simple regression model built on top of linear regression. The R^2 values here are much more reasonable even though they still point to the model not being very accurate (i.e. it only accounts for ~15% of the varriance in the inputs)

In [10]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)

fold = 1
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    
    reg = Ridge().fit(X_train, Y_train)
    
    sklearn_save_model(reg, f"one_hot_normalized_ridge_{fold}")
    
    print(fold, reg.score(X_test, Y_test))
    fold += 1

1 0.15537522031899173
2 0.14645872738475607
3 0.1517421372749565
4 0.1429300413064368
5 0.16822623187486024
6 0.13511006062729075
7 0.14631382963852613
8 0.15032421565709025
9 0.14930621672111155
10 0.15874074480508418


### XGBoost

Finally, I tried XGBoost and used the `mean_squared_error` metric from sklearn to deterimine the efficacy of the model. This seems like by far the best model out of the three.

In [11]:
import xgboost
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter('ignore')

kf = KFold(n_splits=10)

fold = 1
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    
    team_1_model = xgboost.XGBRegressor()
    team_1_model.fit(X_train, Y_train[:,0])
    
    team_1_model.save_model(f"../models/one_hot_normalized_xgboost_team1_{fold}")
    
    Y_pred = team_1_model.predict(X_test)
      
    print(fold, "team 1: ", mean_squared_error(Y_test[:,0], Y_pred))
    
    team_2_model = xgboost.XGBRegressor()
    team_2_model.fit(X_train, Y_train[:,1])
    
    team_2_model.save_model(f"../models/one_hot_normalized_xgboost_team2_{fold}")
    
    Y_pred = team_2_model.predict(X_test)
    
    print(fold, "team 2: ", mean_squared_error(Y_test[:,1], Y_pred))
    
    fold += 1

warnings.simplefilter('default')

1 team 1:  0.022249531112405144
1 team 2:  0.02061860043583991
2 team 1:  0.021839219920246712
2 team 2:  0.018814701483779634
3 team 1:  0.020671732662901395
3 team 2:  0.019776037945074248
4 team 1:  0.022286996164710783
4 team 2:  0.018271522569450024
5 team 1:  0.02337466473925353
5 team 2:  0.01705846511362418
6 team 1:  0.02213388814706473
6 team 2:  0.019432188683494642
7 team 1:  0.022198228114312955
7 team 2:  0.01803622038012955
8 team 1:  0.021163141957751548
8 team 2:  0.019815712234535616
9 team 1:  0.022359866290017284
9 team 2:  0.018962091490736944
10 team 1:  0.021855447541070074
10 team 2:  0.018546629083653614


## With season and location advantage

Next, I added the season and location advantage information to the dataset created above. I normalized the season and just used the `playoff` and `neutral` features as is since they were already binary and contained in \[0, 1\].

In [12]:
from sklearn.preprocessing import MinMaxScaler

season_scaler = MinMaxScaler()
location_games = games.copy()

location_games['norm_season'] = season_scaler.fit_transform(np.array(location_games.season).reshape(-1, 1))

sklearn_save_model(season_scaler, "season_scaler")

location_games.norm_season.describe()

/home/grant/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


count    16274.000000
mean         0.637118
std          0.259674
min          0.000000
25%          0.479592
50%          0.683673
75%          0.846939
max          1.000000
Name: norm_season, dtype: float64

### Preparing data for input to models

To prepared the data for the models I combined all the input features into a numpy array and both the about scores into another numpy array.

In [13]:
from sklearn.utils import shuffle

X = np.array(location_games.team1.tolist())
Z = np.array(location_games.team2.tolist())

X = np.append(X, Z, 1)

X = np.append(X, np.array(location_games.elo1).reshape(-1, 1), 1)
X = np.append(X, np.array(location_games.elo2).reshape(-1, 1), 1)
X = np.append(X, np.array(location_games.norm_season).reshape(-1, 1), 1)
X = np.append(X, np.array(location_games.neutral).reshape(-1, 1), 1)
X = np.append(X, np.array(location_games.playoff).reshape(-1, 1), 1)

Y = np.array(location_games.score1).reshape(-1, 1)
Y = np.append(Y, np.array(location_games.score2).reshape(-1, 1), 1)

X, Y = shuffle(X, Y)

### Linear Regression

As before, I first started with Linear Regression. Although again the R^2 value is all over the place, it's better than the previous combination of features.

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)

fold = 1
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    
    reg = LinearRegression().fit(X_train, Y_train)
    
    sklearn_save_model(reg, f"location_linear_{fold}")
    
    print(fold, reg.score(X_test, Y_test))
    fold += 1

1 -3.6379245568591347e+22
2 -1.1080414665607573e+23
3 -1.0205642794781214e+20
4 -1.4091364352457322e+22
5 -2.1163947670880343e+23
6 -7.533992793545158e+24
7 -1.885816587634706e+23
8 -1.2397478398905725e+22
9 -1.0247396828978012e+20
10 -6.333517506958273e+22


### Ridge

I again used Ridge Regression next. This time we got a slightly better R^2 value of around 0.16-0.17 so this model is performing slightly better than previously.

In [15]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

kf = KFold(n_splits=10)

fold = 1
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    
    reg = Ridge().fit(X_train, Y_train)
    
    sklearn_save_model(reg, f"location_ridge_{fold}")
    
    print(fold, reg.score(X_test, Y_test))
    fold += 1

1 0.17964184500968602
2 0.14897721241050454
3 0.15854515599280622
4 0.15735656391939656
5 0.16315869946947315
6 0.16568047161371924
7 0.16556812933080176
8 0.14792162225025063
9 0.14907187371972838
10 0.158312559207424


### XGBoost

Finally, I again used XGBoost. It's slightly better than the previous XGBoost model with the mean squared error being lower on average.

In [16]:
import xgboost
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter('ignore')

kf = KFold(n_splits=10)

fold = 1
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    
    team_1_model = xgboost.XGBRegressor()
    team_1_model.fit(X_train, Y_train[:,0])
    
    team_1_model.save_model(f"../models/location_xgboost_team1_{fold}")
    
    Y_pred = team_1_model.predict(X_test)
      
    print(fold, "team 1: ", mean_squared_error(Y_test[:,0], Y_pred))
    
    team_2_model = xgboost.XGBRegressor()
    team_2_model.fit(X_train, Y_train[:,1])
    
    team_2_model.save_model(f"../models/location_xgboost_team2_{fold}")
    
    Y_pred = team_2_model.predict(X_test)
    
    print(fold, "team 2: ", mean_squared_error(Y_test[:,1], Y_pred))
    
    fold += 1

warnings.simplefilter('default')

1 team 1:  0.020861266018437706
1 team 2:  0.018309058922310155
2 team 1:  0.02131860875182649
2 team 2:  0.017727997686414372
3 team 1:  0.02116473384242938
3 team 2:  0.01866636375211307
4 team 1:  0.022080999117143528
4 team 2:  0.0175524209421751
5 team 1:  0.021497598188876182
5 team 2:  0.018334304249945945
6 team 1:  0.021977151139225708
6 team 2:  0.018594674682317156
7 team 1:  0.020783183876191314
7 team 2:  0.019785168953806283
8 team 1:  0.020803634178154118
8 team 2:  0.01790700915622672
9 team 1:  0.02184906127909316
9 team 2:  0.01767855805518466
10 team 1:  0.020583902597928118
10 team 2:  0.018517739425069762


## Final Thoughts

For both feature sets, I need to look more into what's going on with Linear Regression since the R^2 values don't make a lot of sense to me. Additionally, I think it could be interesting to look into adding a feature that encapsulates previous match ups of a given two teams. I'm not exactly sure how it would work, maybe an average point delta from all past season (or past `x` seasons). Overall though, I think the Ridge and XGBoost models performed pretty well.